In [27]:
#SPARQL client
import sys
sys.path.append('/home/jovyan')

from Tools.src.sparql_client_smw import SmwSparqlClient
from pprint import pprint

#user: user; pass: 
sparqlClient = SmwSparqlClient("https://graph.onterface.open-semantic-lab.org/blazegraph/namespace/kb/sparql", domain = "onterface.open-semantic-lab.org")#, auth = "basic", user='user', password='')

In [87]:
#query for cells (SPARQL client)
cell_id = "OSL324bc7d8ba754cc9a9cbeb6b082063b4" #https://onterface.open-semantic-lab.org/wiki/Term:OSL324bc7d8ba754cc9a9cbeb6b082063b4
where_statement = f"?subject (^property:IsA)*/(^property:HasProperty)*/(^property:HasParameter)*/(^property:HasPart)*/property:HasId '{cell_id}'" #Cell
res = sparqlClient.get_sparql_triplets(where_statement, debug=False)
pprint(res)

{'property:HasThickness': {'owl:equivalentProperty': ['https://qudt.org/vocab/quantitykind/Thickness',
                                                      'https://qudt.org/vocab/quantitykind/Thickness',
                                                      'https://qudt.org/vocab/quantitykind/Thickness'],
                           'property:Corresponds_to': ['1.000.000.000 nm '
                                                       ',nanometer,nanometers,nanometre,nanometres',
                                                       '1 '
                                                       'm,meter,meters,metre,metres',
                                                       '100 cm '
                                                       ',centimeter,centimeters,centimetre,centimetres',
                                                       '1000 mm '
                                                       ',millimeter,millimeters,millemetre,millimetres',
                           

In [88]:
def osl2batmo(cell_id):
    d = {}
    for key, value in res[cell_id].items():
        for part in value:
            if key == "property:HasParameter": 
                pval = res[part]["property:HasValue"][0].replace('wiki:',"")
                try: pval = float(pval)
                except: pass
                d[res[res[part]["property:HasProperty"][0]]["property:HasBattMoPropertyName"][0]] = pval
            if key == "property:HasPart": 
                d[res[res[part]["property:IsA"][0]]["property:HasBattMoPropertyName"][0]] = osl2batmo(part)
    return d

cell_id = "OSL324bc7d8ba754cc9a9cbeb6b082063b4"  
d = {}
d["geometry"] = osl2batmo("term:" + cell_id)
pprint(d)

{'geometry': {'Electrolyte': {'Separator': {'thickness': 1.5e-05}},
              'NegativeElectrode': {'ActiveMaterial': {'thickness': 6.4e-05}},
              'PositiveElectrode': {'ActiveMaterial': {'thickness': 5.7e-05}},
              'faceArea': 0.001,
              'format': '1D'}}
